In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [2]:
def get_target(df, target_name):
    return df.drop(target_name, axis=1), df[target_name]

def drop_correlated_features(df, threshold):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    df = df.drop(to_drop, axis=1)
    return df

def drop_min_rows(df, *values):
    for value in values:
        df = df.drop(df[df['DP'] == value].index, axis=0)
    return df

def balance_classes(df):
    df_balanced = pd.DataFrame()
    min_class_count = df['DP'].value_counts().min()
    for label in df['DP'].unique():
        df_label = df[df['DP'] == label]
        if len(df_label) > min_class_count:
            df_random_sample = df_label.sample(min_class_count)
            df_balanced = pd.concat([df_balanced, df_random_sample])
        else:
            df_balanced = pd.concat([df_balanced, df_label])
    return df_balanced

In [3]:
from posixpath import split
# load dataset

df = pd.read_csv("Data.csv", header = 0, delimiter = ";")
print("drop duplicates")
df = df.drop_duplicates()
constant_features = [column for column in df.columns if df[column].nunique() == 1]
print("drop constant features = ",len(constant_features))
df.drop(columns=constant_features, inplace=True)
print("split features and target")

df_features, df_target = get_target(df, 'DP')

df_corr = drop_correlated_features(df_features, 0.7)

df = pd.concat([df_corr, df_target], axis=1)
new_df = drop_min_rows(df, -60, -140)

new_df = balance_classes(new_df)

df_check = new_df.copy()

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2

kbest = SelectKBest(k=10, score_func=mutual_info_classif)
kbest.fit(df_check.drop('DP', axis=1), df_check['DP'])
rec = kbest.get_support()
print(rec)
list_best = []
for i, r in enumerate(rec):
    if r == True:
        list_best.append(df_check.columns[i])
        print(df_check.columns[i], kbest.scores_[i])
if 'DP' in list_best:
    list_best.remove('DP')
df_best = df_check[list_best]

# check that dataframe contain column

if df_best.Disbalance.empty == None:
    df_best =  pd.concat([df_check.Disbalance, df_best], axis=1)

df_best.head()

drop duplicates
drop constant features =  1361
split features and target
[ True False False False  True False  True  True False False  True  True
 False False False False False False False False  True False False  True
 False False False False False False False False False False False False
 False False False False False False  True False False False False False
 False  True False False False False False False False False False]
Disbalance 1.170542340666753
PG gen 5 1.1134801745351441
QG gen 27 0.6206451304179863
QN node 7005 0.6690722879140494
U node 7012 0.6431105838941669
QN node 7015 0.568142395887727
PN node 7065 0.7440913093322548
QN node 7073 0.7738759931885291
QN node 7611 0.7014514150690054
QN node 7776 0.792790391659902


,Disbalance,PG gen 5,QG gen 27,QN node 7005,U node 7012,QN node 7015,PN node 7065,QN node 7073,QN node 7611,QN node 7776
5,143.14,0.0,0.0,0.660812,119.229026,0.0,4.448826,8.0,0.0,1.131414
15,142.77,0.0,0.0,0.625786,119.225691,0.0,4.469421,8.0,0.0,1.121359
16,142.60,0.0,0.0,0.703919,119.228304,0.0,4.363729,8.0,0.0,1.121407
18,143.20,0.0,0.0,0.619212,119.240238,0.0,4.375185,8.0,0.0,1.101456
19,142.49,0.0,0.0,0.685193,119.237180,0.0,4.330210,8.0,0.0,1.101435


In [12]:
# data for pytorch model

data_labels = df_check["DP"]
data_features = df_best

# torch tensors

data_features = torch.tensor(data_features.values, dtype=torch.float32)
data_labels = torch.tensor(data_labels.values, dtype=torch.float32)

# split data

train_features, test_features, train_labels, test_labels = train_test_split(
    data_features, data_labels, test_size=0.25, random_state=42
)

# train model

model = LinearRegression()
model.fit(train_features, train_labels)

# test model

test_predictions = model.predict(test_features)

# calculate RMSE

rmse = np.sqrt(mean_squared_error(test_labels, test_predictions))
print(rmse)


5.878734


In [14]:
# pytorch multiclass classification



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Net()
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch, i + 1, running_loss / 2000))
            running_loss = 0.0



print('Finished Training')


Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


NameError: name 'trainloader' is not defined